<a href="https://colab.research.google.com/github/retico/cmepda_medphys/blob/master/L8_code/Lecture8_demo_CAE_semantic_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Convolutional autoencoder (CAE) for semantic segmentation

Semantic Segmentation is a classic Computer Vision problem which involves taking as input some raw data (e.g. 2D or 3D images) and converting them into a mask with regions of interest highlighted.

**Objective**

We will train a Convolutional Auto-Encoder (CAE) to segment mass lesions in mammograms. 

We will use the dataset made available at: 

https://drive.google.com/drive/folders/1-HoF6Qy8lrw4-LANXDL1lOTJTH9XXsDy?usp=sharing

You can either add this folder to your drive ("Add shortcut to drive") or download the large_sample_Im_segmented_ref.zip we wil use, which contains 177 mass examples (104 benign and 73 malignant masses) and the segmentation masks.

**Legend of file names** 

Mass lesions represented in each image.pgm are malignant if the file name ends with “_1.pgm”, benign if it ends with “_2.pgm”, e.g.: 

0007p1_1_1.pgm contains a malignant mass

0003f1_1_1_2.pgm contains a benign mass


**Segmented masses.**

The folders whose name ends with "_ref", e.g. 

small_sample_Im_segmented_ref/ and 

large_sample_Im_segmented_ref/

contain for each original IMAGE or the small or large sample the IMAGE_resized.pgm and the IMAGE_mass_mask.pgm (which has the same size of the IMAGE_resized.pgm).
These lesion segmentation masks have been generated with segmentation code mass_segment.m developed in Lecture5 (cfr. https://github.com/retico/cmepda_medphys)




## Reading data from Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!unzip -q /content/gdrive/My\ Drive/cmepda_medphys_dataset/IMAGES/Mammography_masses/large_sample_Im_segmented_ref.zip -d /content/

##  Dataset overview

In [ ]:
import os
import PIL

In [ ]:
dataset_path = "/content/large_sample_Im_segmented_ref"

We have two kinds of images: *_resized*, i.e. the images containing the mass lesions, and *_mass_mask*, i.e. the lesion masks. 

In [ ]:
!ls /content/large_sample_Im_segmented_ref/ | head -n 4

In [ ]:
PIL.Image.open(os.path.join(dataset_path, "0069p1_4_2_resized.pgm"))

In [ ]:
PIL.Image.open(os.path.join(dataset_path, "0069p1_4_2_mass_mask.pgm"))

## Reading the images in memory

In [ ]:
import glob
import math
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread

In [ ]:
def read_dataset(dataset_path, x_id ="_resized", y_id="_mass_mask"):
    fnames = glob.glob(os.path.join(dataset_path, f"*{x_id}.pgm"  ))
    X = []
    Y = []
    for fname in fnames:
        X.append(imread(fname)[1:,1:,np.newaxis])
        Y.append(imread(fname.replace(x_id, y_id))[1:,1:,np.newaxis])
    return np.array(X), np.array(Y) 

In [ ]:
X,Y = read_dataset(dataset_path)

In [ ]:
print(X.shape, Y.shape)
print(X.min(), X.max(), Y.min(), Y.max())

In [ ]:
X = X/255
Y = Y/255

# Train and test split

We split the dataset in a train and a test sets. 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape)

We will use the *train set* (X_train) to train the model (allowing for an internal train-validation split). We will leave apart the *test set* (X_test) to evaluate the model performance.

# Defining and training the model

We are trying to define a convolutional autoencoder. Inthe paper by (Liu et al, 
*Deep convolutional neural network and 3D deformable approach for tissue segmentation in musculoskeletal Magnetic Resonance Imaging*, Magn Reson Med 2018; 79(4):2379-2391, https://onlinelibrary.wiley.com/doi/10.1002/mrm.26841 there is an example of a possible architecture.

In [ ]:
from keras.layers import Conv2D, Conv2DTranspose, Input
from keras.models import Model, load_model

In [ ]:
def make_model(shape=(124,124,1)):
    input_tensor = Input(shape=shape)
    x = Conv2D(32, (5, 5), strides=2, padding='same', activation='relu')(input_tensor)
    x = Conv2D(64, (3,3), strides=2,  padding='same', activation='relu')(x)
    x = Conv2D(128, (3,3), strides=2, padding='same', activation='relu')(x)

    x = Conv2DTranspose(64, (3,3), strides=2,  padding='same', activation='relu')(x)
    x = Conv2DTranspose(32, (3,3), strides=2, padding='same',activation='relu')(x)
    x = Conv2DTranspose(32, (3,3), strides=2, padding='same',activation='relu')(x)
    out = Conv2D(1, (5,5), padding='valid',activation='sigmoid')(x) #era tanh
    model = Model(input_tensor, out)
    
    return model

In [ ]:
model = make_model()
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['MAE']) 

In [ ]:
history = model.fit(X_train,Y_train, validation_split=0.2, epochs=250)

We can visualize the loss and the MAE on train and validation sets

In [ ]:
import matplotlib.pyplot as plt
plt.semilogy(history.history['loss'])
plt.semilogy(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.figure()
plt.semilogy(history.history['MAE'])
plt.semilogy(history.history['val_MAE'])
plt.legend(['MAE', 'val_MAE'])

We can choose to save the model weights (save_best_only=True) that realized the best performance on the internal validation set (*early stop*)

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    "model.{epoch:02d}-{val_MAE:.4f}.h5", 
    monitor='val_MAE', 
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto', save_freq='epoch')

We build a 'new' model 

In [ ]:
model = make_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['MAE']) 

We need to make an additional train-validation split

In [ ]:
history = model.fit(X_train,Y_train, validation_split=0.2, epochs=200, callbacks=[checkpoint])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.figure()
plt.plot(history.history['MAE'])
plt.plot(history.history['val_MAE'])

In [ ]:
!ls -ltr /content/

We can load a saved model, and evaluate its performance on the test examples

In [ ]:
#!cp /content/gdrive/My\ Drive/cmepda_medphys_dataset/NETS/autoenc_mammo_mass.h5 /content/
model = load_model("/content/model.195-0.0286.h5")
model.summary()

We visualize the output on images of the train set ...

In [ ]:
idx=67
xtrain = X_train[idx][np.newaxis,...]
ytrain = Y_train[idx][np.newaxis,...]
xtrain.shape

In [ ]:
plt.figure(figsize=(14,4))
plt.subplot(1,3,1)
plt.imshow(xtrain.squeeze())
plt.subplot(1,3,2)
plt.imshow(ytrain.squeeze())
plt.subplot(1,3,3)
plt.imshow(model.predict(xtrain).squeeze()>0.2)

and on images of the test set (never seen by the CAE) 

In [ ]:
idx=18
xtest = X_test[idx][np.newaxis,...]
ytest = Y_test[idx][np.newaxis,...]
xtest.shape

In [ ]:
plt.figure(figsize=(14,4))
plt.subplot(1,3,1)
plt.imshow(xtest.squeeze())
plt.subplot(1,3,2)
plt.imshow(ytest.squeeze())
plt.subplot(1,3,3)
plt.imshow(model.predict(xtest).squeeze()>0.2)

# Evaluation of the performance

We can quantify the segmentation performance on the train and test in terms of the Dice index

In [ ]:
def dice(pred, true, k = 1):
    intersection = np.sum(pred[true==k]) * 2.0
    dice = intersection / (np.sum(pred) + np.sum(true))
    return dice

We compute the Dice index for the train examples:

In [ ]:
idx=67

xtrain = X_train[idx][np.newaxis,...]
ytrain = Y_train[idx][np.newaxis,...]
print(Y_train[idx].shape, ytrain.shape)

ypred = model.predict(xtrain).squeeze()>0.2
ytrue = Y_train[idx].squeeze()
print(ypred.shape, ytrue.shape)

In [ ]:
dice_value = dice(ypred, ytrue)
print(dice_value)

We want to compute the Dice index for all images in the np.arrays of the train and test sets

In [ ]:
def dice_vectorized(pred, true, k = 1):
    intersection = 2.0 *np.sum(pred * (true==k), axis=(1,2,3))
    dice = intersection / (pred.sum(axis=(1,2,3)) + true.sum(axis=(1,2,3))) 
    return dice

In [ ]:
dice_vectorized(ytrain,model.predict(xtrain)>0.2)

The average Dice on the train set is:

In [ ]:
dice_vectorized(Y_train,model.predict(X_train)>0.2).mean()

In [ ]:
dice_vectorized(Y_test,model.predict(X_test)>0.2).mean()

You can explore the dependence of the average Dice values on the threshold used to binarize the CAE output

# Data augmentation

If we have a limited number of training examples, as usually happens in medical image analysis, we can artificially augment the dataset by applying transformation to the images and lesions masks. 

We can use the Keras ImageDataGenerator class

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle

In [ ]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='reflect')

In [ ]:
transform = train_datagen.get_random_transform((124,124))
transform
#x = train_datagen.apply_transform(im, transform)
#y = train_datagen.apply_transform(label, transform)


In [ ]:
from tensorflow.keras.utils import Sequence
class MassesSequence(Sequence):
    """ Data augmentation class for a CAE """

    def __init__(self, x, y, img_gen, batch_size=10, shape=(124,124)):
        """ Initialize the sequence

        Parameters:

        x (np.array): images
        y (np.array): labels
        batch_size (int): batch size
        img_gen (ImageDatagenerator): a Keras ImageDataGenerator instance
        shape (tuple): image shape. Default (124, 124)

        """
        self.x, self.y = x, y
        self.shape = shape
        self.img_gen = img_gen
        self.batch_size = batch_size

    def __len__(self):
        return len(self.x) // self.batch_size

    def on_epoch_end(self):
        """Shuffle the dataset at the end of each epoch."""
        self.x, self.y = shuffle(self.x, self.y)

    def process(self, img, transform):
        """ Apply a transformation to an image """
        img = self.img_gen.apply_transform(img, transform)
        return img
            
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
            
        X=[];
        Y=[];
        for image, label in zip(self.x, self.y):
            transform = self.img_gen.get_random_transform(self.shape)
            X.append(self.process(image, transform))
            Y.append(self.process(label, transform)>0.2)

          
        return np.array(X), np.array(Y)

We have at this time to manually split the Train dataset in a Train_tr and a Train_val dataset in order to use an internal validation set during the model.fit. 

We then augment only the Train_tr set and use the Train_val dataset as it is.

In [ ]:
X_train_tr, X_train_val, Y_train_tr, Y_train_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=24)

In [ ]:
mass_gen = MassesSequence(X_train_tr, Y_train_tr, train_datagen)

We can have a look at the mass_gen output

In [ ]:
batch = mass_gen[6]
print(batch[0].shape, batch[1].shape) # they are the data (X) and the masks (Y), respectively

In [ ]:
plt.imshow(np.squeeze(batch[0][1]))
plt.figure()
plt.imshow(np.squeeze(batch[1][1]))

We train the model with augmented data and save the model at the best epoch on the internal validation set

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    "model_augm.{epoch:02d}-{val_MAE:.4f}.h5", 
    monitor='val_MAE', 
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto', save_freq='epoch')

In [ ]:
model = make_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['MAE']) 
history_augm = model.fit(mass_gen, steps_per_epoch=len(mass_gen), epochs=100, validation_data=(X_train_val, Y_train_val), callbacks=[checkpoint])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history_augm.history['loss'])
plt.plot(history_augm.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.figure()
plt.plot(history_augm.history['MAE'])
plt.plot(history_augm.history['val_MAE'])
plt.legend(['MAE', 'val_MAE'])

# Evaluation of the performance of the augmented model

In [ ]:
!ls -ltr /content/

You can load one of the models saved in /content/ folder, e.g the last saved best model on the internal validation set. 

In [ ]:
model = load_model("/content/model_augm.82-0.0255.h5")

I saved a model trained for 250 epochs on this train set (i.e. obtained with a train_test_split fixed random state of '42' in the first Train-Test split and of '24' in the Train_tr-Train_val split).

The saved model with weigths can be found on
https://drive.google.com/drive/folders/1dkv_3J6SbzXu5J1yV-W6uidGDZr1zJxn

As before, you can add a shorthcut to this folder to your drive.

In [ ]:
!cp /content/gdrive/My\ Drive/cmepda_medphys_dataset/NETS/autoenc_mammo_mass_augm_model_augm.137-0.0183.h5 /content/model_augm.137-0.0183.h5

In [ ]:
model = load_model("/content/model_augm.137-0.0183.h5")

Average Dice index on the Train set

In [ ]:
dice_vectorized(Y_train,model.predict(X_train)>0.2).mean()

Average Dice index on the Test set

In [ ]:
dice_vectorized(Y_test,model.predict(X_test)>0.2).mean()

We can have a look at some test images and their segmentations

In [ ]:
idx=30
xtest = X_test[idx][np.newaxis,...]
ytest = Y_test[idx][np.newaxis,...]
plt.figure(figsize=(14,4))
plt.subplot(1,3,1)
plt.imshow(xtest.squeeze())
plt.subplot(1,3,2)
plt.imshow(ytest.squeeze())
plt.subplot(1,3,3)
plt.imshow(model.predict(xtest).squeeze()>0.2)

# To do
*   You can repeat the CAE training by changing the random state in the train_test_split to check the stability of the results against the train-val-test dataset composition.
*   You can explore the impact of using diffeent optimizers (with different parameters) and metrics in the training phase (e.g. MAPE instead of MAE).
* You can change the model parameters. 
* ...

